In [1]:
import pandas as pd
import h2o
h2o.init()
df = pd.read_csv('data/c2.csv')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/ubuntu/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpif7_7cqo
  JVM stdout: /tmp/tmpif7_7cqo/h2o_ubuntu_started_from_python.out
  JVM stderr: /tmp/tmpif7_7cqo/h2o_ubuntu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 1 day
H2O_cluster_name:,H2O_from_python_ubuntu_8k09io
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,29.97 Gb
H2O_cluster_total_cores:,30
H2O_cluster_allowed_cores:,30
H2O_cluster_status:,"locked, healthy"


In [2]:
import numpy as np
df['checking_status'] = df['checking_status'].map({"'<0'": 0, "'0<=X<200'": 200, "'>=200'": 400, "'no checking'": -1})

df['employment'] = df['employment'].map({"unemployed": -1, "'<1'": 1, "'1<=X<4'": 4, "'4<=X<7'": 7, "'>=7'": 10})

df['savings_status'] = df['savings_status'].map({"'<100'": 100, "'100<=X<500'": 500, "'500<=X<1000'": 1000, "'>=1000'": 2000, "'no known savings'": -1})

df['class'] = df['class'].map({'good': 0, 'bad': 1})

df['credit_amount'] = df['credit_amount'].apply(lambda x: np.log(x))
dfh = h2o.H2OFrame(df)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
from sklearn.metrics import average_precision_score
average_precision_score(test.as_data_frame()['class'],grid.models[0].predict(test).as_data_frame()['p1'])

xgboost prediction progress: |

/home/ubuntu/.local/lib/python3.10/site-packages/h2o/frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


███████████████████████████████████████████████████| (done) 100%


0.8047461607827611

In [38]:
aml_atp = h2o.automl.H2OAutoML(max_runtime_secs=600, stopping_metric='aucpr', nfolds=5, sort_metric='aucpr')
aml_atp.train(x=[col for col in train.columns if col != 'class'], y='class', training_frame=train)
lb = aml_atp.leaderboard

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [39]:
lb

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_Best1000_1_AutoML_4_20240121_194416,0.85706,0.929012,0.307665,0.144587,0.307429,0.0945126
StackedEnsemble_AllModels_6_AutoML_4_20240121_194416,0.855983,0.929086,0.30895,0.143168,0.308616,0.0952437
StackedEnsemble_AllModels_3_AutoML_4_20240121_194416,0.851824,0.925679,0.314609,0.153017,0.311692,0.097152
StackedEnsemble_BestOfFamily_6_AutoML_4_20240121_194416,0.848667,0.923452,0.318732,0.148403,0.314372,0.0988299
StackedEnsemble_AllModels_4_AutoML_4_20240121_194416,0.84742,0.925419,0.315982,0.151287,0.311845,0.097247
StackedEnsemble_BestOfFamily_5_AutoML_4_20240121_194416,0.841692,0.917181,0.333427,0.161801,0.322504,0.104009
StackedEnsemble_BestOfFamily_4_AutoML_4_20240121_194416,0.84032,0.918839,0.327632,0.152795,0.317507,0.100811
StackedEnsemble_AllModels_5_AutoML_4_20240121_194416,0.828807,0.918995,0.331992,0.15142,0.316732,0.100319
GBM_lr_annealing_selection_AutoML_4_20240121_194416_select_model,0.824023,0.909431,0.386372,0.18181,0.349997,0.122498
StackedEnsemble_BestOfFamily_3_AutoML_4_20240121_194416,0.81289,0.904159,0.359499,0.177551,0.333287,0.11108


In [40]:
average_precision_score(test.as_data_frame()['class'],h2o.get_model('StackedEnsemble_Best1000_1_AutoML_4_20240121_194416').predict(test).as_data_frame()['p1'])

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


0.8814414487890537

In [41]:
h2o.get_model('StackedEnsemble_Best1000_1_AutoML_4_20240121_194416')

key,value
Stacking strategy,cross_validation
Number of base models (used / total),11/100
# GBM base models (used / total),5/38
# XGBoost base models (used / total),0/16
# DeepLearning base models (used / total),6/43
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [31]:
h2o.get_model('StackedEnsemble_BestOfFamily_7_AutoML_3_20240121_191834')

key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/5
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [30]:
average_precision_score(test.as_data_frame()['class'],h2o.get_model('StackedEnsemble_BestOfFamily_7_AutoML_3_20240121_191834').predict(test).as_data_frame()['p1'])

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


0.8529667573607705

In [35]:
h2o.get_model('GBM_grid_1_AutoML_3_20240121_191834_model_194').actual_params

{'model_id': 'GBM_grid_1_AutoML_3_20240121_191834_model_194',
 'training_frame': 'AutoML_3_20240121_191834_training_py_3_sid_9d22',
 'validation_frame': None,
 'nfolds': 5,
 'keep_cross_validation_models': False,
 'keep_cross_validation_predictions': True,
 'keep_cross_validation_fold_assignment': False,
 'score_each_iteration': False,
 'score_tree_interval': 5,
 'fold_assignment': 'Modulo',
 'fold_column': None,
 'response_column': 'class',
 'ignored_columns': [],
 'ignore_const_cols': True,
 'offset_column': None,
 'weights_column': None,
 'balance_classes': False,
 'class_sampling_factors': None,
 'max_after_balance_size': 5.0,
 'max_confusion_matrix_size': 20,
 'ntrees': 54,
 'max_depth': 16,
 'min_rows': 1.0,
 'nbins': 20,
 'nbins_top_level': 1024,
 'nbins_cats': 1024,
 'r2_stopping': 1.7976931348623157e+308,
 'stopping_rounds': 0,
 'stopping_metric': 'AUCPR',
 'stopping_tolerance': 0.02486823656509969,
 'max_runtime_secs': 54.94625,
 'seed': 2872821691682241183,
 'build_tree_one_